# Data Cleaning & Merging

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib
import itertools as it

In [2]:
census_data = pd.read_csv("census_data_merged.csv")
var_data = pd.read_excel("All Coded Variables.xlsx")


In [3]:
census_data.head(5)

,Unnamed: 0,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,black_population,total_population,cty_percentage
0,1,Alabama,G0100010,41.0,1,10,Autauga,NaN,253472.0,1327718.0,0.190908
1,2,Alabama,G0100030,41.0,1,30,Baldwin,NaN,910598.0,6044606.0,0.150646
2,3,Alabama,G0100050,41.0,1,50,Barbour,NaN,302774.0,1023340.0,0.295868
3,4,Alabama,G0100070,41.0,1,70,Bibb,NaN,2139712.0,4241136.0,0.504514
4,5,Alabama,G0100090,41.0,1,90,Blount,NaN,113314.0,4467024.0,0.025367


In [4]:
census_data.isna().sum()

Unnamed: 0             0
STATE                  0
GISJOIN                0
STATEICP              92
STATEFIPS              0
COUNTYFIPS             0
ALLCOUNTIES            0
VARIABLE NAME       3258
black_population     208
total_population     208
cty_percentage       208
dtype: int64

<p> We have no data for Puerto Rico in our original dataset, so I will use the estimate of the Black alone population (0.10) found here: https://www.census.gov/quickfacts/fact/table/PR 
    </p>

In [5]:
census_data.loc[census_data['STATE']=='Puerto Rico','cty_percentage'] = 0.10


In [6]:
missing_pcts = census_data.loc[census_data['cty_percentage'].isna() == True]['STATE'].unique()

for state in missing_pcts: 
    black_pop = census_data.loc[census_data['STATE'] == state]['black_population'].sum()
    tot_pop = census_data.loc[census_data['STATE'] == state]['total_population'].sum()
    state_mean = black_pop/tot_pop
    missing_data = census_data.loc[census_data['STATE'] == state,'cty_percentage'].isna()
    row_ind = missing_data.loc[missing_data == True].index
    
    for i in row_ind:
        census_data.iloc[i,len(census_data.columns)-1] = state_mean
    


/var/folders/l9/1jc9yfg14f5gdn2jw_yyb7100000gn/T/ipykernel_33163/2299198326.py:6: RuntimeWarning: invalid value encountered in double_scalars
  state_mean = black_pop/tot_pop


<p> Manually imputing the value for DC based on the value in the "Data_Merging" file; not sure why it's missing here. Also imputing the "Alaska Territory" values with the mean value for Alaska. 

In [7]:
census_data.loc[census_data['STATE'] == 'District Of Columbia', 'cty_percentage'] = 0.480405
census_data.loc[census_data['STATE'] == 'Alaska Territory', 'cty_percentage'] = census_data.loc[census_data['STATE'] == 'Alaska']['cty_percentage'].mean()

In [8]:
census_data.isna().sum()

Unnamed: 0             0
STATE                  0
GISJOIN                0
STATEICP              92
STATEFIPS              0
COUNTYFIPS             0
ALLCOUNTIES            0
VARIABLE NAME       3258
black_population     208
total_population     208
cty_percentage         0
dtype: int64

In [9]:
var_data.head(5)


,Unnamed: 0,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,GR.SV2,GR.LRA2,GR.LRA3,MM.LRA1,EP.SV1,EP.LRA1,EP.SV2,EP.LRA2,EP.SV3
0,1,Alabama,G0100010,41.0,1,10,Autauga,1,1,1,1,0,0,0,0,1
1,2,Alabama,G0100030,41.0,1,30,Baldwin,1,1,1,1,0,0,0,1,0
2,3,Alabama,G0100050,41.0,1,50,Barbour,1,1,1,1,0,0,0,0,0
3,4,Alabama,G0100070,41.0,1,70,Bibb,1,1,1,1,0,0,0,0,0
4,5,Alabama,G0100090,41.0,1,90,Blount,1,1,1,1,0,0,0,0,0


In [10]:
var_data.isna().sum()

Unnamed: 0      0
STATE           0
GISJOIN         0
STATEICP       92
STATEFIPS       0
COUNTYFIPS      0
ALLCOUNTIES     0
GR.SV2          0
GR.LRA2         0
GR.LRA3         0
MM.LRA1         0
EP.SV1          0
EP.LRA1         0
EP.SV2          0
EP.LRA2         0
EP.SV3          0
dtype: int64

In [11]:
missing_data = census_data.loc[census_data['STATE'] == 'Alaska','cty_percentage'].isna()
row_ind = missing_data.loc[missing_data == True].index
for i in row_ind:
    print(census_data.iloc[i,])
#census_data[row_ind,]

In [12]:
data = var_data.merge(census_data)
data.isna().sum()

Unnamed: 0             0
STATE                  0
GISJOIN                0
STATEICP              76
STATEFIPS              0
COUNTYFIPS             0
ALLCOUNTIES            0
GR.SV2                 0
GR.LRA2                0
GR.LRA3                0
MM.LRA1                0
EP.SV1                 0
EP.LRA1                0
EP.SV2                 0
EP.LRA2                0
EP.SV3                 0
VARIABLE NAME       3242
black_population     192
total_population     192
cty_percentage         0
dtype: int64

In [13]:
data = data.drop(columns = ["STATEICP", "VARIABLE NAME", "black_population", "total_population", "Unnamed: 0"])
data.isna().sum()

STATE             0
GISJOIN           0
STATEFIPS         0
COUNTYFIPS        0
ALLCOUNTIES       0
GR.SV2            0
GR.LRA2           0
GR.LRA3           0
MM.LRA1           0
EP.SV1            0
EP.LRA1           0
EP.SV2            0
EP.LRA2           0
EP.SV3            0
cty_percentage    0
dtype: int64

In [14]:
data.to_csv("full_data_final.csv")